# Test Teacher Model (Ollama)

This notebook tests the connection to the local Ollama server and verifies the teacher model (gemma3-27b-it-qat:latest) is working correctly.

In [ ]:
import ollama
import time
import json
from typing import Dict, Any

## Test Basic Connection

In [ ]:
def test_ollama_connection():
    """Test basic connection to Ollama server"""
    try:
        # List available models
        models = ollama.list()
        print("Available models:")
        for model in models['models']:
            print(f"  - {model['name']}")
        return True
    except Exception as e:
        print(f"Error connecting to Ollama: {e}")
        return False

# Test connection
connection_ok = test_ollama_connection()

## Test Teacher Model

In [ ]:
TEACHER_MODEL = "gemma3-27b-it-qat:latest"

def query_teacher_model(prompt: str, model: str = TEACHER_MODEL) -> Dict[str, Any]:
    """Query the teacher model with error handling"""
    try:
        start_time = time.time()
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={
                'temperature': 0.1,  # Low temperature for consistent responses
                'top_p': 0.9,
                'num_predict': 100  # Limit response length for testing
            }
        )
        end_time = time.time()
        
        return {
            'success': True,
            'response': response['response'],
            'duration': end_time - start_time,
            'model': model
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'model': model
        }

# Test simple query
test_prompt = "What is 2 + 2? Please answer with just the number."
result = query_teacher_model(test_prompt)

print(f"Query result: {result}")

## Test Needle-in-Haystack Task

In [ ]:
def test_needle_task():
    """Test the teacher model on a simple needle-in-haystack task"""
    
    # Create a simple test case
    haystack = "The quick brown fox jumps over the lazy dog. NEEDLE_STRING is here. More random text follows."
    needle = "NEEDLE_STRING"
    
    prompt = f"""Find the starting position (character index) of the string "{needle}" in the following text:

{haystack}

Please respond with just the number (the character index where "{needle}" starts)."""
    
    result = query_teacher_model(prompt)
    
    if result['success']:
        print(f"Teacher response: {result['response']}")
        print(f"Duration: {result['duration']:.2f} seconds")
        
        # Verify the correct answer
        correct_position = haystack.find(needle)
        print(f"Correct position: {correct_position}")
    else:
        print(f"Error: {result['error']}")
    
    return result

# Run the test
needle_result = test_needle_task()

## Summary

This notebook verifies:
1. Connection to Ollama server
2. Teacher model availability and response quality
3. Basic needle-in-haystack task performance

The teacher model will be used to generate training data for the MARU architecture.